In [12]:
import pandas as pd

# Load Excel file
file_path = "C:/Users/riyac/Downloads/Analytics Position Case Study.xlsx"
xls = pd.ExcelFile(file_path)


In [13]:
# Load all sheets
gameplay_df = xls.parse("User Gameplay data", skiprows=3)
deposit_df = xls.parse("Deposit Data", skiprows=3)
withdrawal_df = xls.parse("Withdrawal Data", skiprows=3)

# Preview
print("Gameplay Data:")
print(gameplay_df.head())

print("Deposit Data:")
print(deposit_df.head())

print("Withdrawal Data:")
print(withdrawal_df.head())


Gameplay Data:
   User ID  Games Played             Datetime
0      851             1  2022-01-10 00:00:00
1      717             1  2022-01-10 00:00:00
2      456             1  2022-01-10 00:00:00
3      424             1  2022-01-10 00:00:00
4      845             1  2022-01-10 00:00:00
Deposit Data:
   User Id             Datetime  Amount
0      357  2022-01-10 00:03:00    2000
1      776  2022-01-10 00:03:00    2500
2      492  2022-01-10 00:06:00    5000
3      803  2022-01-10 00:07:00    5000
4      875  2022-01-10 00:09:00    1500
Withdrawal Data:
   User Id             Datetime  Amount
0      190  2022-01-10 00:03:00    5872
1      159  2022-01-10 00:16:00    9540
2      164  2022-01-10 00:24:00     815
3      946  2022-01-10 00:29:00   23000
4      763  2022-01-10 00:40:00    9473


In [14]:
# Rename columns for clarity
gameplay_df.columns = ["User_ID", "Games_Played", "Datetime"]
deposit_df.columns = ["User_ID", "Datetime", "Amount"]
withdrawal_df.columns = ["User_ID", "Datetime", "Amount"]

# Convert datetime columns
gameplay_df["Datetime"] = pd.to_datetime(gameplay_df["Datetime"])
deposit_df["Datetime"] = pd.to_datetime(deposit_df["Datetime"])
withdrawal_df["Datetime"] = pd.to_datetime(withdrawal_df["Datetime"])


In [15]:
# Gameplay points
gameplay_points = gameplay_df.groupby("User_ID")["Games_Played"].sum().reset_index()
gameplay_points["Gameplay_Points"] = gameplay_points["Games_Played"] * 1
gameplay_points.drop(columns="Games_Played", inplace=True)

# Deposit points
deposit_points = deposit_df.groupby("User_ID")["Amount"].sum().reset_index()
deposit_points["Deposit_Points"] = deposit_points["Amount"] * 0.01
deposit_points.drop(columns="Amount", inplace=True)

# Withdrawal points
withdrawal_points = withdrawal_df.groupby("User_ID")["Amount"].sum().reset_index()
withdrawal_points["Withdrawal_Points"] = withdrawal_points["Amount"] * 0.005
withdrawal_points.drop(columns="Amount", inplace=True)


In [16]:
# Merge all data
user_points = pd.merge(gameplay_points, deposit_points, on="User_ID", how="outer")
user_points = pd.merge(user_points, withdrawal_points, on="User_ID", how="outer")

# Fill NaNs with 0
user_points.fillna(0, inplace=True)

# Total points
user_points["Total_Points"] = user_points["Gameplay_Points"] + user_points["Deposit_Points"] + user_points["Withdrawal_Points"]

# Final output
user_points.sort_values("Total_Points", ascending=False).head()


,User_ID,Gameplay_Points,Deposit_Points,Withdrawal_Points,Total_Points
634,634,24,5150.00,78688.525,83862.525
765,765,24097,0.00,155.000,24252.000
99,99,10,11648.00,12015.705,23673.705
672,672,10,21587.00,1168.750,22765.750
212,212,1,19249.81,2949.250,22200.060


In [17]:
# Save the result to Excel or CSV
user_points.to_csv("user_reward_points.csv", index=False)
print("Saved as 'user_reward_points.csv'")


Saved as 'user_reward_points.csv'
